In [1]:
import pandas as pd

In [2]:
store_info = pd.read_csv("업체정보.csv")

In [3]:
store_info.head()

,업체명,지역,업종,주소,전화번호,썸네일
0,강군초밥,충주시,"초밥,롤",충북 충주시 연원로 28 1층 강군초밥,043-845-3412,https://search.pstatic.net/common/?autoRotate=...
1,장수골,충주시,한식,충북 충주시 대소원면 쇠실로 984,043-846-4189,https://search.pstatic.net/common/?autoRotate=...
2,원조영덕가든,충주시,한식,충북 충주시 노은면 감노로 1595-1 원조영덕가든,043-853-2264,https://search.pstatic.net/common/?autoRotate=...
3,충주댐198510,충주시,"카페,디저트",충북 충주시 충주호수로 586,0507-1456-1985,https://search.pstatic.net/common/?autoRotate=...
4,비건채식뷔페 빛가람,충주시,"채식,샐러드뷔페",충북 충주시 신대2길 26 비건채식뷔페 빛가람,043-842-3333,https://search.pstatic.net/common/?autoRotate=...


In [4]:
store_info['업체명'] = store_info['업체명'] + ' ' + store_info['지역']

In [5]:
store_info.head()

,업체명,지역,업종,주소,전화번호,썸네일
0,강군초밥 충주시,충주시,"초밥,롤",충북 충주시 연원로 28 1층 강군초밥,043-845-3412,https://search.pstatic.net/common/?autoRotate=...
1,장수골 충주시,충주시,한식,충북 충주시 대소원면 쇠실로 984,043-846-4189,https://search.pstatic.net/common/?autoRotate=...
2,원조영덕가든 충주시,충주시,한식,충북 충주시 노은면 감노로 1595-1 원조영덕가든,043-853-2264,https://search.pstatic.net/common/?autoRotate=...
3,충주댐198510 충주시,충주시,"카페,디저트",충북 충주시 충주호수로 586,0507-1456-1985,https://search.pstatic.net/common/?autoRotate=...
4,비건채식뷔페 빛가람 충주시,충주시,"채식,샐러드뷔페",충북 충주시 신대2길 26 비건채식뷔페 빛가람,043-842-3333,https://search.pstatic.net/common/?autoRotate=...


In [6]:
store_info_review = pd.read_csv("업체정보_리뷰(여기다합쳐).csv", encoding='cp949')

In [7]:
store_info_review.head()

,업체명,리뷰
0,장수골 충주시,"토종닭누룽지백숙 먹었는데 토종닭 크고 살도 쫀득쫀 득 맛있습니다. 잡내, 비린내 없..."
1,장수골 충주시,장소가 넓어서 매번 가족모임을 하게 되는데 이곳은 골프장 손님들이 대부분이더라고요 ...
2,장수골 충주시,해천탕에 해산물도 다양하고 양도 많습니다 !! 닭은 부드럽고 문어도 쫄깃하니 맛있었...
3,장수골 충주시,밑반찬이 맛있고 특히 닭백숙 다먹고 나오는 \n누룽지 닭백숙이 정말 맛있어서 가족모...
4,장수골 충주시,정말 맛있게먹었습니다! 맛집이라고 예약해서 간 집입니다^^


In [8]:
store_names = set(store_info['업체명'])

# store_info_review에서 업체명 추출
review_store_names = set(store_info_review['업체명'])

# 누락된 업체명 찾기
missing_names = store_names - review_store_names



In [9]:
len(missing_names)

89

In [10]:
df = pd.DataFrame(missing_names, columns=['업체명'])

In [11]:
df.to_csv('missing_names.csv', index=False)

In [43]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# CSV 파일 읽기
source = pd.read_csv('missing_names.csv')

# 업체명과 지역을 결합하여 검색어 리스트 생성
keywords = [f"{row['업체명']}" for idx, row in source.iterrows()]

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# 네이버 지도 페이지 열기
driver.get('https://map.naver.com/')

# 결과를 저장할 리스트 생성
results = []

for keyword in keywords:
    try:
        # 검색창 대기 및 입력
        search = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".input_search"))
        )

        # 검색창 초기화 (이전 검색어 삭제)
        search.clear()

        # 검색어 입력 및 검색
        search.send_keys(keyword)
        search.send_keys(Keys.ENTER)

        # 프레임 전환 대기 및 프레임 전환 (entryIframe)
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe#entryIframe"))
        )

        # 리뷰 탭 클릭 대기 및 클릭
        try:
            review_tab = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, "a.tpj9w._tab-menu[role='tab'][aria-selected='false']"))
            )
            review_tab.click()

            # 4번째 탭 선택
            try:
                target_tab = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, "a.tpj9w._tab-menu[role='tab'][aria-selected='false']:nth-of-type(6)"))
                )
                target_tab.click()

                # 최대 100개의 리뷰 수집
                reviews_collected = 0
                while True:
                    try:
                        more_button = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.fvwqf"))
                        )
                        driver.execute_script("arguments[0].scrollIntoView();", more_button)
                        more_button.click()
                        time.sleep(2)  # 클릭 후 대기 시간

                        # 스크롤 다운
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                        # 새로 수집한 리뷰 개수 확인
                        new_reviews = len(driver.find_elements(By.CSS_SELECTOR, "span.zPfVt"))
                        reviews_collected += new_reviews

                        # 최대 100개까지만 수집
                        if reviews_collected >= 100:
                            break

                    except Exception as e:
                        print(f"더보기 버튼이 더 이상 없음: {str(e)}")
                        break

                # 모든 리뷰 내용과 방문일 추출
                reviews = driver.find_elements(By.CSS_SELECTOR, "span.zPfVt")
                visit_dates = driver.find_elements(By.CSS_SELECTOR, "span.place_blind")

                review_texts = []
                visit_dates_texts = []

                for review, visit_date in zip(reviews, visit_dates):
                    review_text = review.text
                    visit_date_text = visit_date.text
                    review_texts.append(review_text)
                    visit_dates_texts.append(visit_date_text)

                # 결과를 리스트에 추가
                results.extend(list(zip([keyword]*len(reviews), review_texts, visit_dates_texts)))

            except Exception as e:
                print(f"4번째 탭 선택 중 에러 발생: {str(e)}")

        except Exception as e:
            print(f"리뷰 탭 클릭 중 에러 발생: {str(e)}")

        # 검색창 기본 프레임으로 전환
        driver.switch_to.default_content()

    except Exception as e:
        print(f"에러 발생: {str(e)}")

    finally:
        # 현재 창과 탭 모두 닫기
        if len(driver.window_handles) > 1:
            for handle in driver.window_handles[1:]:
                driver.switch_to.window(handle)
                driver.close()
            driver.switch_to.window(driver.window_handles[0])

        # 검색창 초기화 (다음 가게로 넘어가기 전에)
        search.clear()

# 데이터프레임 생성
df_result = pd.DataFrame(results, columns=['업체명', '리뷰', '방문일'])

# 데이터프레임 출력
print(df_result)

# 브라우저 닫기
driver.quit()


4번째 탭 선택 중 에러 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x00D7C1C3+27395]
	(No symbol) [0x00D13DC4]
	(No symbol) [0x00C11B7F]
	(No symbol) [0x00C52C65]
	(No symbol) [0x00C52D3B]
	(No symbol) [0x00C8EC82]
	(No symbol) [0x00C739E4]
	(No symbol) [0x00C8CB24]
	(No symbol) [0x00C73736]
	(No symbol) [0x00C47541]
	(No symbol) [0x00C480BD]
	GetHandleVerifier [0x01033A93+2876371]
	GetHandleVerifier [0x01087F5D+3221661]
	GetHandleVerifier [0x00DFD634+556916]
	GetHandleVerifier [0x00E0474C+585868]
	(No symbol) [0x00D1CE04]
	(No symbol) [0x00D19818]
	(No symbol) [0x00D199B7]
	(No symbol) [0x00D0BF0E]
	BaseThreadInitThunk [0x77267BA9+25]
	RtlInitializeExceptionChain [0x77ADC10B+107]
	RtlClearBits [0x77ADC08F+191]

더보기 버튼이 더 이상 없음: Message: 
Stacktrace:
	GetHandleVerifier [0x00D7C1C3+27395]
	(No symbol) [0x00D13DC4]
	(No symbol) [0x00C11B7F]
	(No symbol) [0x00C52C65]
	(No symbol) [0x00C52D3B]
	(No symbol) [0x00C8EC82]
	(No symbol) [0x00C739E4]
	(No symbol) [0x00C8CB24]
	(No symbol) [0x00C73736]


In [44]:
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

def remove_illegal_characters(text):
    if isinstance(text, str):
        return ILLEGAL_CHARACTERS_RE.sub('', text)
    return text

In [45]:
df_result = df_result.applymap(remove_illegal_characters)

# 데이터프레임을 엑셀 파일로 저장
df_result.to_excel('업체정보_리뷰_3.xlsx', index=False, engine='openpyxl')

df_result.to_csv('업체정보_리뷰_3.csv', index=False)

C:\Users\sande\AppData\Local\Temp\ipykernel_1492\3606352039.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_result = df_result.applymap(remove_illegal_characters)
